In [1]:
import pandas as pd
import string
import numpy as np
import math

In [5]:
A = pd.read_csv('../../datasets/sample_citeseer_100k.csv')
B = pd.read_csv('../../datasets/sample_dblp_100k.csv')
l_block_attr = 'title'
r_block_attr = 'title'
ltable = A
rtable = B
lid = 'id'
rid = 'id'

In [21]:
def remove_stopwords(tokens, stopwords):
    out_tokens = []
    for token in tokens:
        if not stopwords.has_key(token):
            out_tokens.append(token)
    return out_tokens
#----------
def process_column(column, stop_words):
    column = column.str.translate(None, string.punctuation)
    column = column.str.lower()
    if stop_words:
        dict_stopwords = dict(zip(self.stop_words, [0] * len(self.stop_words)))
        partial_rm_stopwords_fn = partial(remove_stopwords,
                                          stopwords=dict_stopwords)
        column = column.str.split().map(partial_rm_stopwords_fn).str.join(' ')
    return column
#----------
def sample_ltable(ltable, lid, l_block_attr, nbins, sample_proportion, seed=0, stopwords=[]):
    ltbl = ltable[[lid, l_block_attr]]
    ltbl = ltbl[~ltbl[l_block_attr].isnull()]
    #process columns
    ltbl[l_block_attr] = process_column(ltbl[l_block_attr], [])
    n = int(math.floor(sample_proportion*len(ltable)))
        
    # get the string lengths
    ltbl['str_len'] = ltbl.title.str.len()
    groups = ltbl.groupby('str_len')
    len_ids = {}
    for gid, g in groups:
        len_ids[gid] = list(g[lid].values)
    strlens = list(ltbl['str_len'].values)
    strlens += [max(strlens)+1]
    
    freq, edges = np.histogram(strlens, bins=nbins)

    
    bins = [[] for _ in range(nbins)]
    keys = sorted(len_ids.keys())
    positions = np.digitize(keys, edges)
    
    for i in range(len(keys)):
        k, p = keys[i], positions[i]
        bins[p-1].extend(len_ids[k])
    len_bins = [len(bins[i]) for i in range(len(bins))]

    
    weights = [len_bins[i]/float(sum(len_bins)) for i in range(len(bins))]
    numtups = [int(math.ceil(weights[i]*n)) for i in range(len(weights))]
    numtups, sum(numtups)
    sampled = []
    for i in range(len(bins)):
        nt = numtups[i]
        np.random.seed(seed)
        if len(bins[i]):
            tmp = np.random.choice(bins[i], nt)
            if len(tmp):
                sampled.extend(tmp)
    ltable.set_index(lid, inplace=True, drop=False)
    return ltable.loc[sampled]
    

In [36]:
    import dmagellan
    from dmagellan.utils.py_utils.utils import build_inv_index, tokenize_strings
    from dmagellan.tokenizer.whitespacetokenizer import WhiteSpaceTokenizer
    from dmagellan.utils.cy_utils.stringcontainer import StringContainer
    from dmagellan.blocker.overlap.overlapblocker import OverlapBlocker


In [37]:
ob = OverlapBlocker()

In [27]:
nbins = 10
sample_proportion = 0.1
s_ltable = sample_ltable(ltable, lid, l_block_attr, nbins, sample_proportion)

10004


[154773, 958200, 625919, 1308726, 904416, 1032734, 29223, 961373, 200980]

In [49]:
tok = WhiteSpaceTokenizer()
ob = OverlapBlocker()
p = ob.process_and_tokenize_ltable(ltable, 'id', 'title', tok, [])
inv_index = build_inv_index([p])

In [51]:
tok

In [83]:
# tbl = rtable[[lid, l_block_attr]]
# tbl = tbl[~tbl[r_block_attr].isnull()]
# #process columns
# tbl[r_block_attr] = process_column(tbl[r_block_attr], [])
# n = int(math.floor(sample_proportion*len(rtable)))
ob = OverlapBlocker()
tok = WhiteSpaceTokenizer()
rtable.reset_index(inplace=True, drop=True)
rtable['_pos'] = list(range(len(rtable)))


x = ob.process_and_tokenize_ltable(rtable, 'id', 'title', tok, [])
d = {}
t = {}
for i in range(x.size()):
    tid, tokens = x.get(i)
    cnt = 0
    for tok in tokens:
        if tok not in t:
            t[tok] = len(inv_index.values(tok))            
        cnt += t[tok]
    d[tid] = cnt

In [84]:
df = pd.DataFrame(d.items(), columns=['id', 'count'])
groups = df.groupby('count')
cnt_ids = {}
for gid, g in groups:
    cnt_ids[gid] = list(g[lid].values)


In [85]:
cnts = list(df['count'].values)
cnts += [max(cnts)+1]
nbins = 10    
freq, edges = np.histogram(cnts, bins=nbins)
seed = 0

In [86]:
sample_proportion = 0.1
n = int(math.floor(sample_proportion*len(ltable)))
bins = [[] for _ in range(nbins)]
keys = sorted(cnt_ids.keys())
positions = np.digitize(keys, edges)
    
for i in range(len(keys)):
    k, p = keys[i], positions[i]
    bins[p-1].extend(cnt_ids[k])
len_bins = [len(bins[i]) for i in range(len(bins))]

    
weights = [len_bins[i]/float(sum(len_bins)) for i in range(len(bins))]
numtups = [int(math.ceil(weights[i]*n)) for i in range(len(weights))]

sampled = []
for i in range(len(bins)):
    nt = numtups[i]
    np.random.seed(seed)
    if len(bins[i]):
        tmp = np.random.choice(bins[i], nt)
        if len(tmp):
            sampled.extend(tmp)
rtable.set_index(rid, inplace=True, drop=False)
s_rtable = rtable.loc[sampled]

In [88]:
g = s_rtable.sort_values('_pos')

,id,title,authors,journal,month,year,publication_type,_pos
id,,,,,,,,
1709973,1709973,Introduction to the special issue on object-ba...,"Fernando Pereira, Shih-Fu Chang, Rob Koenen, A...",IEEE Trans. Circuits Syst. Video Techn.,NaN,1999,article,1
2065604,2065604,Two-sided matching with interdependent values.,"Archishman Chakraborty, Alessandro Citanna, Mi...",J. Economic Theory,NaN,2010,article,21
1044095,1044095,MuSoft: Multimedia in der Softwaretechnik.,"Klaus Alfert, Ernst-Erich Doberkat, Gregor Eng...",NaN,NaN,2003,inproceedings,24
581644,581644,Dogear: Social bookmarking in the enterprise.,"David R. Millen, Jonathan Feinberg, Bernard Kerr",NaN,NaN,2006,inproceedings,51
1393173,1393173,TCP: Thread Contention Predictor for Parallel ...,"Aparna Mandke Dani, Bharadwaj Amrutur, Y. N. S...",NaN,NaN,2012,inproceedings,54


In [75]:
len(s_rtable)

10005

In [77]:
A.reset_index(inplace=True, drop=True)

In [78]:
tt = zip(A.index.values, A['id'].values)

In [81]:
ttd = pd.DataFrame(dict(tt).items(), columns=['position', 'id'])

In [82]:
ttd.head()

,position,id
0,0,937228
1,1,752870
2,2,307594
3,3,156173
4,4,870908
